In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import warnings

sys.path.append('../src/')
warnings.filterwarnings("ignore")

from server import Server
from utils import plot_exp

### First experiment : impact of federated learning

In [ ]:
LR = 0.01
EPOCHS = 5
NR_TRAINING_ROUNDS = 100
BATCH_SIZE = 128

RANGE_NR_CLIENTS = [1, 5, 10]

In [ ]:
for nr_clients in RANGE_NR_CLIENTS:
    print(f"### Number of clients : {nr_clients} ###\n\n")
    server = Server(
        nr_clients=nr_clients,
        nr_training_rounds=NR_TRAINING_ROUNDS,
        data='MNIST',
        epochs=EPOCHS,
        lr=LR,
        batch_size=BATCH_SIZE,
        is_private=False,
        epsilon=None,
        max_grad_norm=None,
        noise_multiplier=None,
        is_parallel=True,
        device=None,
        verbose='all')

    server.train(early=True, patience=2)


### Second experiment : impact of differential privacy

In [ ]:
NR_CLIENTS = 5
NR_TRAINING_ROUNDS = 100
EPOCHS = 1
PATIENCE = 20
LR = 0.01
BATCH_SIZE = 8

MAX_GRAD_NORM = 1.2
NOISE_MULTIPLIER = None

RANGE_EPSILON = [100]

In [ ]:
experiment_losses, experiment_accs = [], []
for epsilon in RANGE_EPSILON:
    print(f"### epsilon : {epsilon} ###\n\n")
    server = Server(
        nr_clients=NR_CLIENTS,
        nr_training_rounds=NR_TRAINING_ROUNDS,
        data='MNIST',
        epochs=EPOCHS,
        lr=LR,
        batch_size=BATCH_SIZE,
        is_private=True,
        epsilon=epsilon,
        max_grad_norm=MAX_GRAD_NORM,
        noise_multiplier=NOISE_MULTIPLIER,
        is_parallel=False,
        device=None,
        verbose='all')

    test_losses, test_accs = server.train(early=False, patience=PATIENCE)
    experiment_losses.append(test_losses)
    experiment_accs.append(test_accs)

In [ ]:
names = [f'ε = {i}' for i in RANGE_EPSILON]
plot_exp(experiment_losses, experiment_accs, names)